# Simulación del modelo

Ejecutando este código se pueden reproducir las simulaciones de las que se proveen los datos o explorar otras simulaciones

In [1]:
from model import *
import _pickle as pickle
from timeit import default_timer as timer
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os


def abrir(file):
    with open(file, 'rb') as input:
        data = pickle.load(input)[0]
    print(data[0])
    return data


def media(datos):
    return datos.sum(axis=0) / len(datos)


def cal_inflation(prices):
    infla = list()

    for i in range(len(prices) - 1):

        if i < len(prices) - 1:
            if prices[i] != 0:
                x = (prices[i + 1] - prices[i]) / prices[i]
            else:
                x = (prices[i + 1] - prices[i - 1]) / prices[i - 1]

            infla.append(x)

    return infla


def time_series(data):
    if len(data) == 3:

        S1, S2 = data[1], data[2]

    else:
        S1, S2 = [data[0]], [data[1]]

    S1_mprices = np.array([sector.m_prices for sector in S1])
    S2_mprices = np.array([sector.m_prices for sector in S2])
    S1_inflation = np.array([cal_inflation(sector.m_prices) for sector in S1])
    S2_inflation = np.array([cal_inflation(sector.m_prices) for sector in S2])
    S1_mr = np.array([sector.m_r for sector in S1])
    S1_H = np.array([sector.herfin for sector in S1])
    S2_H = np.array([sector.herfin for sector in S2])
    S1_numfirms = np.array([sector.num_firms for sector in S1])
    S2_numfirms = np.array([sector.num_firms for sector in S2])

    series_S1 = {'prices': media(S1_mprices), 'r': media(S1_mr), 'Herfindhal': media(S1_H),
                 'Firm number': media(S1_numfirms), 'Inflation': np.append(np.nan, media(S1_inflation))}
    series_S2 = {'prices': media(S2_mprices), 'Herfindhal': media(S2_H), 'Firm number': media(S2_numfirms),
                 'Inflation': np.append(np.nan, media(S2_inflation))}

    d1, d2 = pd.DataFrame(series_S1), pd.DataFrame(series_S2)
    d1, d2 = d1.round(5), d2.round(5)

    return d1, d2


def save( file_name, data, directory):

    df1, df2 = time_series(data)

    working = os.getcwd()
    os.chdir(directory)

    writer = pd.ExcelWriter(file_name)
    df1.to_excel(writer, 'Sector 1')
    df2.to_excel(writer, 'Sector 2')
    writer.save()

    os.chdir(working)

def save_xls(list_dfs, xls_path):
    writer = pd.ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer,'run%s' % n)
    writer.save()
    writer.close()

def save_all(data, directory, filename):
    simulations = data
    data_frames_S1 = []
    data_frames_S2 = []

    for e, i in zip(simulations[1], simulations[2]):
        a, b = time_series([e, i])
        data_frames_S1.append(a)
        data_frames_S2.append(b)

    save_xls(data_frames_S1, directory + filename + 'S1.xlsx')
    save_xls(data_frames_S2, directory + filename + 'S2.xlsx')

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, -1)

def simu_save(n_1, n_2, N,T, confi = 'normal'):
    capital = []
    consumption = []
    for i in range(N):
        S1, S2 = simulate(n_1, n_2, T, spec = confi)
        print(i)
        capital.append(S1)
        consumption.append(S2)
        spec = {'Specification': confi, 'Runs': N, 'Range': T, 'Firms': [n_1,n_2]}

    return spec, capital, consumption

keys = ["normal", "mark I", "mark II"]


simulations.append(simu_save(1,1,100,5000, confi = 'mark I'))


saving = '/Users/'


save_object(simulations, "markI_multi.pkl")
save_all(simulations[0], saving_2, '/markI_100')
save('markI_100.xlsx', simulations[0], saving_2)







ModuleNotFoundError: No module named 'model'